In [1]:
import cobra

In [2]:
import numpy, pandas, pickle

# 0. user-defined variables

In [4]:
model_file = '/Users/adrian/projects/riia/data/model/Recon3DModel_301.mat'
results_file = '/Users/adrian/projects/riia/results/manual_ko.pickle'
threads = 8

In [5]:
cobra_config = cobra.Configuration()
cobra_config.processes = threads
cobra.Configuration()

Attribute,Description,Value
solver,Mathematical optimization solver,glpk
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,8
cache_directory,Path for the model cache,/Users/adrian/Library/Caches/cobrapy
max_cache_size,Maximum cache size in bytes,104857600
cache_expiration,Model cache expiration time in seconds (if any),None


# 1. read model

In [6]:
model = cobra.io.load_matlab_model(model_file)
model.optimize()
model.objective.value

755.0032155506631

# 2. running single KO first

In [7]:
%%time
print(len(model.genes))
single_ko_results = cobra.flux_analysis.single_gene_deletion(model, model.genes)

2248
CPU times: user 1.43 s, sys: 307 ms, total: 1.73 s
Wall time: 16.6 s


In [8]:
print(single_ko_results.shape)
print(numpy.min(single_ko_results['growth']), numpy.max(single_ko_results['growth']))
singledf = single_ko_results.sort_values(by=['growth'])
singledf.head(10)

(2248, 3)
0.0 755.0032155506635


,ids,growth,status
1751,{54675.1},0.000000,optimal
2022,{1468.1},663.411530,optimal
582,{8802.1},674.472254,optimal
1269,{10632.1},714.843470,optimal
1450,{27109.1},714.843470,optimal
1318,{539.1},714.843470,optimal
875,{521.1},714.843470,optimal
433,{267020.1},714.843470,optimal
1422,{4905.1},714.843470,optimal
1765,{8803.1},714.843470,optimal


In [9]:
singledf[singledf['ids'] == set({'5836.1'})]['growth']

193    755.003216
Name: growth, dtype: float64

In [10]:
# goi = set({'5836.1'})
# singledf[singledf['ids'] == goi]

In [11]:
# for reaction in model.reactions:
#     possibilities = reaction.gene_reaction_rule.split('or')
#     if len(possibilities) == 1:
#         print(reaction.id)
#         print(len(reaction.genes), reaction.genes)
#         print(reaction.gene_reaction_rule, type(reaction.gene_reaction_rule))
#         print()

# 3. manual implementation of single KOs

In [12]:
%%time

results = {}
for working_gene in model.genes:
    #print(working_gene)
    
    #! iterate reactions
    with model as model:
        for reaction in model.reactions:
            local_reaction_genes = [gene.id for gene in reaction.genes]
            
            #! if the reaction contains the gene...
            if working_gene.id in local_reaction_genes:
                #print(local_reaction_genes)
                possibilities = reaction.gene_reaction_rule.split('or')
                #print(possibilities)
                if possibilities == 1: #! the gene is in the reaction, therefore the minimum is 1, not 0
                    reaction.bounds = (0, 0)
                    
        #! optimize
        model.optimize()
        #print(working_gene, model.objective.value)
        results[working_gene] = model.objective.value

CPU times: user 1min 59s, sys: 226 ms, total: 2min
Wall time: 2min


In [13]:
# convert results dictionary into pandas dataframe
df = pandas.DataFrame.from_dict(results, columns=['Objective value'], orient='index')
manualdf = df.sort_values(by=['Objective value'])
manualdf.head(10)

,Objective value
314.2,755.003216
51251.1,755.003216
100526794.1,755.003216
84618.1,755.003216
22978.1,755.003216
80347.1,755.003216
10587.1,755.003216
1727.1,755.003216
51700.1,755.003216
51706.1,755.003216


# 3. compare singleKO and manual results

In [19]:
for gene in manualdf.index:
    #print(gene)
    
    # single KO values
    #gene='54675.1'
    single_KO_value = singledf[singledf['ids'] == set({'{}'.format(gene)})]['growth'].values[0]
    #print(single_KO_value)
    
    # manual KO values
    #print(manualdf.loc[gene]['Objective value'])
    manual_KO_value = manualdf.loc[gene]['Objective value']
    
    # difference
    diff = single_KO_value - manual_KO_value
    
    # if difference larger than threshold
    if diff > 1e-12: # 1e-13 will give you some prints
        print(gene)
        print(single_KO_value)
        print(manual_KO_value)
        print(diff)
        print()
    
    #print()